<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.-Composite-Types-(II)" data-toc-modified-id="4.-Composite-Types-(II)-1">4. Composite Types (II)</a></span><ul class="toc-item"><li><span><a href="#4.4-Structs" data-toc-modified-id="4.4-Structs-1.1">4.4 Structs</a></span><ul class="toc-item"><li><span><a href="#Recursive-Data-Structures" data-toc-modified-id="Recursive-Data-Structures-1.1.1">Recursive Data Structures</a></span></li><li><span><a href="#Struct-Literals" data-toc-modified-id="Struct-Literals-1.1.2">Struct Literals</a></span></li><li><span><a href="#Comparing-Structs" data-toc-modified-id="Comparing-Structs-1.1.3">Comparing Structs</a></span></li><li><span><a href="#Struct-Embedding-and-Anonymous-Fields" data-toc-modified-id="Struct-Embedding-and-Anonymous-Fields-1.1.4">Struct Embedding and Anonymous Fields</a></span></li></ul></li><li><span><a href="#4.5-JSON" data-toc-modified-id="4.5-JSON-1.2">4.5 JSON</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#github-HTTP-request" data-toc-modified-id="github-HTTP-request-1.2.0.1">github HTTP request</a></span></li><li><span><a href="#github-api-for-issues" data-toc-modified-id="github-api-for-issues-1.2.0.2">github api for issues</a></span></li></ul></li></ul></li><li><span><a href="#4.6-Text-and-HTML-Templates" data-toc-modified-id="4.6-Text-and-HTML-Templates-1.3">4.6 Text and HTML Templates</a></span></li></ul></li></ul></div>

# 4. Composite Types (II)
*    **Structs**
     *    Struct Literals
     *    Comparing Structs
     *    Struct Embedding and Anonymous Fields
*    JSON
*    Text and HTML Templates


## 4.4 Structs

* A struct is an aggregate data type grouping together zero or more named values of arbitrary types as a single entity
* Each value is called a field
* The name of a struct field is exported if it begins with a capital letter; this is Go's main access control mechanism
* A struct type may contain a mixture of exported and unexported fields

<code>
type struct_name struct {
  member1 datatype;
  member2 datatype;
  member3 datatype;
  ...
}
</code>

In [ ]:
//Employee
import "fmt"
import "time"
type Employee struct {
    ID int
    Name string
    Address string
    DoB time.Time //date of birth
    Position string
    Salary int
    ManagerID int
}
func main(){
    
    alan := Employee{ID:1, ManagerID: 3, Position: "Vice President"}
    dilbert := Employee{ID:2, ManagerID: 1}
      
    employeeDB := make(map[int]*Employee)
    employeeDB[alan.ID] = &alan 
    employeeDB[dilbert.ID] = &dilbert
    
    fmt.Printf("%q\n",employeeDB)
    for _,emp := range employeeDB {
        fmt.Printf("%q\n",emp)
        fmt.Println((*emp).ID, emp.Position)
    }
}

In [ ]:
//Employee
import "fmt"
import "time"
type Employee struct {
    ID int
    Name string
    Address string
    DoB time.Time
    Position string
    Salary int
    ManagerID int
}
func main(){
    
    alan := Employee{ID:1, ManagerID: 3, Position: "Vice President"}
    dilbert := Employee{ID:2, ManagerID: 1}
    dilbert.Salary -= 5000  // demoted, for writing too few lines of code
    position := &dilbert.Position
    *position = "Senior" + *position //promoted, for outsourcing to Elbonia
    
    var employeeOfTheMonth *Employee = &dilbert
    employeeOfTheMonth.Position += " (proactive team player)"
    //(*employeeOfTheMonth).Position += " (proactive team player)"
    // employeeOfTheMonth.Position and (*employeeOfTheMonth).Position are equivalent
    
    employeeDB := make(map[int]*Employee)
    employeeDB[alan.ID] = &alan
    employeeDB[dilbert.ID] = &dilbert
    
    fmt.Printf("%q\n",employeeDB)
    for _,emp := range employeeDB {
        fmt.Printf("%q\n",emp)
    }
}

In [ ]:
import "fmt"
var employeeDB map[int]*Employee
//func EmployeeByID(id int) Employee { return *employeeDB[id]}
func EmployeeByID(id int) *Employee { return employeeDB[id]}

func main() {
    alan := Employee{ID:1, ManagerID: 3, Position: "Vice President"}
    employeeDB = make(map[int]*Employee)
    employeeDB[alan.ID] = &alan
    dilbert := Employee{ID:2, ManagerID: 1}
    employeeDB[dilbert.ID] = &dilbert
    
    fmt.Println(EmployeeByID(dilbert.ManagerID).Position) // "boss"
    id := dilbert.ID    
    EmployeeByID(id).Salary = 0  //dilert is fired
                                 //Should return *Employee, to be in left-hand-side
}

### Recursive Data Structures
* Struc type ***S*** can't declare a field of the same type ***S***
  * aggregate value cannot containt itself
* S may declare a field of the pointer type \****S***
  * create recursive data structure like lists and trees
* Implement an insertion sort in the following

In [ ]:
//gopl.io/ch4/treesort
import ("math/rand";"sort";"fmt")  
type tree struct {
	value       int
	left, right *tree
}

// Sort sorts values in place.
func Sort(values []int) {
	var root *tree
	for _, v := range values {
		root = add(root, v)
	}
	appendValues(values[:0], root) // append in the slice, the first position
    //appendValues(values, root)     // append in the last position
    //appendValues(values[:], root)    // append in the last position
}

// appendValues appends the elements of t to values in order
// and returns the resulting slice.
func appendValues(values []int, t *tree) []int {
    //fmt.Println(values,"len",len(values),cap(values))
	if t != nil {
		values = appendValues(values, t.left)
		values = append(values, t.value)
		values = appendValues(values, t.right)
	}
	return values
}

func add(t *tree, value int) *tree {
	if t == nil {
		// Equivalent to return &tree{value: value}.
		t = new(tree)
		t.value = value
		return t
	}
	if value < t.value {
		t.left = add(t.left, value)
	} else {
		t.right = add(t.right, value)
	}
	return t
}

func main(){
    data := make([]int, 50)
    for i := range data {
        data[i] = rand.Int() % 50
    }
    
	//treesort.Sort(data)
    Sort(data)
//sort.Ints(data)
    if !sort.IntsAreSorted(data) {
		fmt.Printf("not sorted: %v", data)
    } else {
        fmt.Printf("sorted: %v", data)
    }
}
//!-

<strike>$go get gopl.io/ch4/treesort</strike>
```
$go build gopl.io/ch4/treesort
$go install gopl.io/ch4/treesort
$go test gopl.io/ch4/treesort
```

In [ ]:
!go test gopl.io/ch4/treesort -v

### Struct Literals

* A value of a struct type can be written using a struct literal specifying values for its fields
* There are two forms of struct literal
* The first form requires a value be specified for every field, in the right order
* The second form is used, in which a struct value is initialized by listing some or all of the field names and their corresponding values
* The two forms cannot be mixed in the same literal

In [ ]:
import "image/gif"
type Point struct{ X, Y int}
func main(){
  p := Point{1,2} // the first form
  q := Point{X:1, Y:2} // the second form
  fmt.Println(p,q)
  
  nframes := 30
  anim := gif.GIF{LoopCount: nframes}
  fmt.Println(anim)
}

In [ ]:
package p
type T struct{ a, b int} // a and b are not exported
package main
import "p"
func main(){
var _ = T{a:1, b:2} //compile error, can't reference a  and b 
}

In [ ]:
//Struct values can be passed as arguments to functions and returned from them
type Point struct{ X, Y int}
func Scale(p Point, factor int) Point {
    return Point{p.X * factor, p.Y * factor}
}

%%
fmt.Println(Scale(Point{1,2},5)) // {5,10}

In [ ]:
//Larger struct types are passsed to or returned from functions indirectly using a pointer
//and this is required if the function must modify the argument
import "time"
type Employee struct {
    ID int
    Name string
    Address string
    DoB time.Time
    Position string
    Salary int
    ManagerID int
}
func Bonus(e *Employee, percent int) int {
    return e.Salary * percent / 100
}
func AwardAnnualRaise(e *Employee) {
    e.Salary = e.Salary * 105 / 100
}

In [ ]:
//two are equivalent
type Point struct{ X, Y int}
%%
pp := &Point{1,2}
fmt.Println(pp)


In [ ]:
type Point struct{ X, Y int}
%%
pp := new(Point)
*pp = Point{1,2}
fmt.Println(pp)

### Comparing Structs
* If all the fields of a struct are comparable, the struct is comparable
* Two expressions of the comparable types may be compared using == or !!=

In [ ]:
//comparing two points
type point struct{X, Y int}
%%
p := Point{1,2}
q := Point{2,1}
fmt.Println(p.X == q.X && p.Y == q.Y) // false
fmt.Println(p == q)                   // false

* Comparable struct types, may be used as the key type of a map

In [ ]:
//comparable struct types as map key
type address struct {
    hostname string
    port     int
}
%%
hits := make(map[address]int)
hits[address{"golang.org",443}]++
hits[address{"gopl.io",80}]+= 23
fmt.Println(hits)

### Struct Embedding and Anonymous Fields
* a convenient syntactic shortcut
* use x.f stands for x.d.e.f

In [ ]:
type Circle struct {
    X,Y, Radius int
}
type Wheel struct {
    X,Y, Radius, Spokes int
}
%%
var w Wheel
w.X = 8
w.Y = 8
w.Radius = 5
w.Spokes = 20

* Factor out their common parts

In [ ]:
import "fmt"
type Point struct {
    X, Y int
}
type Circle struct {
    Center Point
    Radius int
}
type Wheel struct {
    Circle Circle
    Spokes int
}
%%
var w Wheel
w.Circle.Center.X = 8
w.Circle.Center.Y = 8
w.Circle.Radius = 5
w.Spokes = 20
fmt.Println(w)

* Declare a field with a type but no name
* Such fields are called anonymous fields
* Due to embedding, we can refer to the names at the leaves of the implicit tree without giving the intervening names

In [ ]:
//access the anonymous field
type Circle struct {
    Point
    Radius int
}
type Wheel struct {
    Circle 
    Spokes int
}
%%
var w Wheel
w.X = 8    //equivalent to w.Circle.Point.X = 8
w.Y = 8    //equivalent to w.Circle.Point.Y = 8
w.Radius = 5 //equivalent to w.Circle.Radius = 5
w.Circle.Radius = 5
w.Spokes = 20 
fmt.Println(w)

* There's no corresponding shorthand for the struct literal syntax

In [ ]:
%%
w := Wheel{8,8,5,20} 
_ = w

In [ ]:
%%
w := Wheel{X:8, Y:8, Radius: 5, Spokes: 20}
_ = w

In [ ]:
### Use one of the two forms

In [ ]:
//Anonymous Fields
type Point struct {
    X, Y int
}
type Circle struct {
    Point
    Radius int
}
type Wheel struct {
    Circle 
    Spokes int
}
%%
var w Wheel
w = Wheel{Circle{Point{8, 8}, 5}, 20}

w = Wheel{
		Circle: Circle{
			Point:  Point{X: 8, Y: 8},
			Radius: 5,
		},
		Spokes: 20, // NOTE: trailing comma necessary here (and at Radius)
}

//fmt.Printf("%#v\n", w)
//%v	the value in a default format
//	when printing structs, the plus flag (%+v) adds field names
//%#v	a Go-syntax representation of the value
fmt.Printf("%+v\n",w)
	// Output:
	// Wheel{Circle:Circle{Point:Point{X:8, Y:8}, Radius:5}, Spokes:20}

w.X = 42

fmt.Printf("%#v\n", w)
fmt.Printf("%+v\n", w)
	// Output:
	// Wheel{Circle:Circle{Point:Point{X:42, Y:8}, Radius:5}, Spokes:20}
	//!-

*  "anonymous" fields have implicit names, we can't have two anonymous fields of the same type
* w.X = 8 //equivalent to w.circle.point.X = 8, but forbidden outside the package

## 4.5 JSON
* JavaScript Object Notation (JSON) is a standard notation for sending and receiving structured information
* Other similar notation: XML, ASN.1, and Google's Protocol Buffers
* API: encoding/json, encoding/xml, encoding/asn1
* basic JSON types are numbers(decimal or scientific notation), booleans(true, false), and strings (sequene of Unicode code points)
* basic JSON types can be combined recursively using JSON arrays and objects
* JSON array is an ordered sequence of values, written as a comma-separated list enclosed in square brackets
* JSON array are used to encode Go arrays and slices
* JSON object is a mapping from strings to values, written as a sequence of name:value pairs separated by commas and surrounded by braces
* JSON objects are used to encode GO maps and structs

Example

type   |value
-------|-----------------------------------
boolean|true
number | -273.15
string |"She said \"Hello, 世界\""
array  | ["gold", "silver", "bronze"]
object |{"year": 1980,<br>"event": "archery",<br>"medals":["gold","silver","brone"]} 

* Gather movie reviews and offer recommendations
* String literals after the Year and Color field are field tags
* Converting Go data structure to JSON by **json.Marshal**
* Only exported fields are marshaled (so, capitalized names for all the Go field names)
* a field tag is a string of metadata associated at compile time with the field of a struct (**Year** changed to **released**)
* Field tag specifies an alternative JSON name for the Go field
* JSON name: total_count, Go field name: TotalCount
* option: no JSON output if the field has the zero value of the type
<code>
  Year int `json:"released"` 
  Color bool `json:"color, omitempty"`
</code>

In [91]:
//gopl.pl/ch4/movie
import ("encoding/json";"fmt";"log")

type Movie struct {
	Title  string
	Year   int  `json:"released"`
	Color  bool `json:"color,omitempty"`
	Actors []string
}

var movies = []Movie{
	{Title: "Casablanca", Year: 1942, Color: false,
		Actors: []string{"Humphrey Bogart", "Ingrid Bergman"}},
	{Title: "Cool Hand Luke", Year: 1967, Color: true,
		Actors: []string{"Paul Newman"}},
	{Title: "Bullitt", Year: 1968, Color: true,
		Actors: []string{"Steve McQueen", "Jacqueline Bisset"}},
	// ...
}

In [92]:
func main(){		//!+Marshal
		data, err := json.Marshal(movies)
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
}		//!-Ma

[{"Title":"Casablanca","released":1942,"Actors":["Humphrey Bogart","Ingrid Bergman"]},{"Title":"Cool Hand Luke","released":1967,"color":true,"Actors":["Paul Newman"]},{"Title":"Bullitt","released":1968,"color":true,"Actors":["Steve McQueen","Jacqueline Bisset"]}]


* json.MarshalIndent produces neatly indented output
* two arguments: 
  * a prefix for each line of output 
  * a string for each level of indentation
* Inverse operation to marshalling: **json.Unmarshal**

In [93]:
//MarshalIndent	
func main (){
		//!+MarshalIndent
		data, err := json.MarshalIndent(movies, "", "    ")
		if err != nil {
			log.Fatalf("JSON marshaling failed: %s", err)
		}
		fmt.Printf("%s\n", data)
		//!-MarshalIndent

		//!+Unmarshal
		var titles []struct{ Title string 
                            Year   int  `json:"released"`
                            Color  bool `json:"color,omitempty"`
                             Actors []string
                           }
		if err := json.Unmarshal(data, &titles); err != nil {
			log.Fatalf("JSON unmarshaling failed: %s", err)
		}
		fmt.Println(titles) // "[{Casablanca} {Cool Hand Luke} {Bullitt}]"
		//!-Unmarshal
}

[
    {
        "Title": "Casablanca",
        "released": 1942,
        "Actors": [
            "Humphrey Bogart",
            "Ingrid Bergman"
        ]
    },
    {
        "Title": "Cool Hand Luke",
        "released": 1967,
        "color": true,
        "Actors": [
            "Paul Newman"
        ]
    },
    {
        "Title": "Bullitt",
        "released": 1968,
        "color": true,
        "Actors": [
            "Steve McQueen",
            "Jacqueline Bisset"
        ]
    }
]
[{Casablanca 1942 false [Humphrey Bogart Ingrid Bergman]} {Cool Hand Luke 1967 true [Paul Newman]} {Bullitt 1968 true [Steve McQueen Jacqueline Bisset]}]


#### github HTTP request 
* decode the result as JSON
* url.QueryEscape to escape \? and \&

In [94]:
//gopl.io/ch4/github
import "time"

const IssuesURL = "https://api.github.com/search/issues"

type IssuesSearchResult struct {
	TotalCount int `json:"total_count"`
	Items      []*Issue
}

type Issue struct {
	Number    int
	HTMLURL   string `json:"html_url"`
	Title     string
	State     string
	User      *User
	CreatedAt time.Time `json:"created_at"`
	Body      string    // in Markdown format
}

type User struct {
	Login   string
	HTMLURL string `json:"html_url"`
}


#### github api for issues
* [github issue api](https://api.github.com/search/issues?q=repo:golang/go%20is:open%20json%20decoder)
* struct fields are capitalized for public access even if JSON names are not
* matching process with JSON names and Go struct names is case-insensitive

In [95]:
import ("encoding/json";"fmt";"net/http";"net/url";"strings")

// SearchIssues queries the GitHub issue tracker.
func SearchIssues(terms []string) (*IssuesSearchResult, error) {
	q := url.QueryEscape(strings.Join(terms, " "))
	resp, err := http.Get(IssuesURL + "?q=" + q)
	if err != nil {
		return nil, err
	}
	if resp.StatusCode != http.StatusOK {
		resp.Body.Close()
		return nil, fmt.Errorf("search query failed: %s", resp.Status)
	}
    //fmt.Println(resp.Body)
    fmt.Printf("%q",resp.Body)
	var result IssuesSearchResult
	if err := json.NewDecoder(resp.Body).Decode(&result); err != nil {
		resp.Body.Close()
		return nil, err
	}
	resp.Body.Close()
	return &result, nil
}

In [96]:
import ("fmt";"log";"os";"gopl.io/ch4/github")
%args repo:golang/go is:open json decoder
func main() {
	result, err := github.SearchIssues(os.Args[1:])
    //t := []string{"repo:golang/go","is:open","json","decoder"}
    //result, err := github.SearchIssues(t)
    //result, err := SearchIssues(t) 
	if err != nil {
		log.Fatal(err)
	}
	fmt.Printf("%d issues:\n", result.TotalCount)
	for i, item := range result.Items {
		fmt.Printf("%d #%-5d %9.9s %.55s\n",i,
			item.Number, item.User.Login, item.Title)
	}
}

87 issues:
0 #48298     dsnet encoding/json: add Decoder.DisallowDuplicateFields
1 #11046     kurin encoding/json: Decoder internally buffers full input
2 #36225     dsnet encoding/json: the Decoder.Decode API lends itself to m
3 #56733 rolandsho encoding/json: add (*Decoder).SetLimit
4 #61627    nabice x/tools/gopls: The rename command line may accept ident
5 #40128  rogpeppe proposal: encoding/json: garbage-free reading of tokens
6 #29035    jaswdr proposal: encoding/json: add error var to compare  the 
7 #41144 alvaroale encoding/json: Unmarshaler breaks DisallowUnknownFields
8 #42571     dsnet encoding/json: clarify Decoder.InputOffset semantics
9 #5901        rsc encoding/json: allow per-Encoder/per-Decoder registrati
10 #40127  rogpeppe encoding/json: add Encoder.EncodeToken method
11 #14750 cyberphon encoding/json: parser ignores the case of member names
12 #43716 ggaaooppe encoding/json: increment byte counter when using decode
13 #34543  maxatome encoding/json: Unmarshal & jso

## 4.6 Text and HTML Templates
* the simplest formatting: Printf
* separate the format from the code 
* **text/template** and **html/template** packages provide a mechanism for substituting the values of variables into a text or HTML template 

* template is a string or file containing one or more portions enclosed in double braces, {{...}}, called actions
* Each action contains an expression in the template language
  * Printing values
  * Selecting struct fields
  * Calling functions and methods
  * Control flow such as if-else, range, and instantiating other templates

In [97]:
// template
import "fmt"
const templ = `{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}`
%%
fmt.Println(templ)

{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}


In [98]:
//Parse the template into a suitable internal represntation
import ("time";"text/template";"os")
func daysAgo(t time.Time) int {
    return int(time.Since(t).Hours()/24)
}
%%
report, err := template.New("report").
   Funcs(template.FuncMap{"daysAgo": daysAgo}).
   Parse(templ)
if (err != nil) {
    fmt.Fprintln(os.Stderr,"error")
} else {
    fmt.Printf("%q\n",report)
}

&{"report" %!q(*parse.Tree=&{report report 0x1400007ed50 0 {{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}} [] <nil> [{8 206  7} {11 24 {{ 2} {0 0  0}] 1 [] map[] 0 0}) %!q(*template.common=&{map[report:0x14000070200] {{0 0} 0 0 0 0} {0} {{0 0} 0 0 0 0} map[daysAgo:0x100df2d50] map[daysAgo:{0x100e60340 0x100e8cd08 19}]}) "" ""}


In [99]:
// Template fixed at compile time and template.Must handle the error condition
import ("log";"os";"text/template";"time";"gopl.io/ch4/github")
const templ = `{{.TotalCount}} issues:
{{range .Items}}----------------------------------------
Number: {{.Number}}
User:   {{.User.Login}}
Title:  {{.Title | printf "%.64s"}}
Age:    {{.CreatedAt | daysAgo}} days
{{end}}`

func daysAgo(t time.Time) int {
	return int(time.Since(t).Hours() / 24)
}

var report = template.Must(template.New("issuelist").
	Funcs(template.FuncMap{"daysAgo": daysAgo}).
	Parse(templ))

%args repo:golang/go is:open json decoder  
func main() {
	result, err := github.SearchIssues(os.Args[1:])
    //result, err := SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
    fmt.Printf("%v\n",result)
	if err := report.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}


&{87 [0x140002200e0 0x14000220150 0x140002201c0 0x14000220230 0x140002202a0 0x14000220310 0x14000220380 0x140002203f0 0x14000220460 0x14000220540 0x140002205b0 0x14000220620 0x14000220690 0x14000220700 0x14000220770 0x140002207e0 0x14000220850 0x140002208c0 0x14000220930 0x140002209a0 0x14000220a10 0x14000220a80 0x14000220af0 0x14000220b60 0x14000220bd0 0x14000220c40 0x14000220cb0 0x14000220d20 0x14000220d90 0x14000220e00]}
87 issues:
----------------------------------------
Number: 48298
User:   dsnet
Title:  encoding/json: add Decoder.DisallowDuplicateFields
Age:    775 days
----------------------------------------
Number: 11046
User:   kurin
Title:  encoding/json: Decoder internally buffers full input
Age:    3065 days
----------------------------------------
Number: 36225
User:   dsnet
Title:  encoding/json: the Decoder.Decode API lends itself to misuse
Age:    1405 days
----------------------------------------
Number: 56733
User:   rolandshoemaker
Title:  encoding/json: add (*Deco

In [100]:
// not handle error, if parse error, exceptions occur
%args repo:golang/go is:open json decoder
func main() {
	//!+parse
	report, err := template.New("report").
		Funcs(template.FuncMap{"daysAgo": daysAgo}).
		Parse(templ)
	if err != nil {
		log.Fatal(err)
	}
	//!-parse
	result, err := github.SearchIssues(os.Args[1:])
    //result, err := SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := report.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}


87 issues:
----------------------------------------
Number: 48298
User:   dsnet
Title:  encoding/json: add Decoder.DisallowDuplicateFields
Age:    775 days
----------------------------------------
Number: 11046
User:   kurin
Title:  encoding/json: Decoder internally buffers full input
Age:    3065 days
----------------------------------------
Number: 36225
User:   dsnet
Title:  encoding/json: the Decoder.Decode API lends itself to misuse
Age:    1405 days
----------------------------------------
Number: 56733
User:   rolandshoemaker
Title:  encoding/json: add (*Decoder).SetLimit
Age:    344 days
----------------------------------------
Number: 61627
User:   nabice
Title:  x/tools/gopls: The rename command line may accept identifiers in
Age:    89 days
----------------------------------------
Number: 40128
User:   rogpeppe
Title:  proposal: encoding/json: garbage-free reading of tokens
Age:    1203 days
----------------------------------------
Number: 29035
User:   jaswdr
Title:  propos

* html/template use the same API and expression language as text/template
* with HTML, JavaScript, CSS, or URLs

In [101]:
//gopl.io/ch4/issueshtml
import ("log";"os";"gopl.io/ch4/github";"html/template")
var issueList = template.Must(template.New("issuelist").Parse(`
<h1>{{.TotalCount}} issues</h1>
<table border=1>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>
{{range .Items}}
<tr>
  <td><a href='{{.HTMLURL}}'>{{.Number}}</a></td>
  <td>{{.State}}</td>
  <td><a href='{{.User.HTMLURL}}'>{{.User.Login}}</a></td>
  <td><a href='{{.HTMLURL}}'>{{.Title}}</a></td>
</tr>
{{end}}
</table>
`))

In [102]:
%args repo:golang/go is:open json decoder

func main() {
	//result, err := SearchIssues(os.Args[1:])
    result, err := github.SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := issueList.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}


<h1>87 issues</h1>
<table border=1>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/48298'>48298</a></td>
  <td>open</td>
  <td><a href='https://github.com/dsnet'>dsnet</a></td>
  <td><a href='https://github.com/golang/go/issues/48298'>encoding/json: add Decoder.DisallowDuplicateFields</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/11046'>11046</a></td>
  <td>open</td>
  <td><a href='https://github.com/kurin'>kurin</a></td>
  <td><a href='https://github.com/golang/go/issues/11046'>encoding/json: Decoder internally buffers full input</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/36225'>36225</a></td>
  <td>open</td>
  <td><a href='https://github.com/dsnet'>dsnet</a></td>
  <td><a href='https://github.com/golang/go/issues/36225'>encoding/json: the Decoder.Decode API lends itself to misuse</a></td>
</tr>

<tr>
  <td><a href='https:

* [issues.html](http://skhuang.github.io/ch4/issues.html)
<code>
$ go build gopl.io/ch4/issueshtml
$ ./issueshtml repo:golang/go commenter:gopherbot json encoder >issues.html
</code>

In [103]:
//os.Args = []string{"issuehtml","repo:golang/go","3133","10535"}
%args repo:golang/go 3133 10535
func main() {
	//result, err := SearchIssues(os.Args[1:])
    result, err := github.SearchIssues(os.Args[1:])
	if err != nil {
		log.Fatal(err)
	}
	if err := issueList.Execute(os.Stdout, result); err != nil {
		log.Fatal(err)
	}
}


<h1>3 issues</h1>
<table border=1>
<tr style='text-align: left'>
  <th>#</th>
  <th>State</th>
  <th>User</th>
  <th>Title</th>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/3133'>3133</a></td>
  <td>closed</td>
  <td><a href='https://github.com/ukai'>ukai</a></td>
  <td><a href='https://github.com/golang/go/issues/3133'>html/template: escape xmldesc as &amp;lt;?xml</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/10535'>10535</a></td>
  <td>open</td>
  <td><a href='https://github.com/dvyukov'>dvyukov</a></td>
  <td><a href='https://github.com/golang/go/issues/10535'>x/net/html: void element &lt;link&gt; has child nodes</a></td>
</tr>

<tr>
  <td><a href='https://github.com/golang/go/issues/47148'>47148</a></td>
  <td>closed</td>
  <td><a href='https://github.com/dmitshur'>dmitshur</a></td>
  <td><a href='https://github.com/golang/go/issues/47148'>all: Go 1.15.14 release status</a></td>
</tr>

</table>


[issues2.html](https://skhuang.github.io/ch4/issues2.html)
<code>
$ ./issueshtml repo:golang/go 3133 10535 >issues2.html
</code>

In [ ]:
!./ch4/issueshtml/issueshtml repo:golang/go 3133 10535

In [ ]:
//gopl.io/ch4/autoescape
import ("fmt";"html/template";"log";"os")

//!+
func main() {
	const templ = `<p>A: {{.A}}</p><p>B: {{.B}}</p>`
	t := template.Must(template.New("escape").Parse(templ))
	var data struct {
		A string        // untrusted plain text, will be escaped 
		B template.HTML // trusted HTML, preseve the markup
	}
    fmt.Printf("%T\n",data)
    data.A = "<b>Hello!</b>"
    data.B ="<b>Hello!</b>"
    //var  b template.HTML= template.HTML("test")
	if err := t.Execute(os.Stdout, data); err != nil {
		log.Fatal(err)
	}
}


[autoescape.html](http://skhuang.github.io/ch4/autoescape.html)

* for more information: 
<code>
$ go doc text/template
$ go doc html/template
</code>

In [ ]:
!go doc text/template

In [ ]:
!go doc html/template